In [1]:
from PIL import Image
import os
import random
import math
import numpy as np
import imgaug as ia
import imgaug.augmenters as iaa
from imgaug.augmentables.segmaps import SegmentationMapsOnImage

In [2]:
base_path = "./roboflow_datasets/runaway-seg-12/train"
destination = "./cropped_roboflow4-224"
crop_size = 224
train = 0.7
val = 0.15
test = 0.15

In [3]:
seq = iaa.Sequential([
    iaa.Fliplr(0.5),  # Horizontal flip with 50% probability
    iaa.Flipud(0.5),  # Vertical flip with 50% probability
    iaa.Affine(
        rotate=(-45, 45),  # Rotate between -45 and 45 degrees
    ),
    iaa.OneOf([  # Either contrast normalization or color augmentation
        iaa.ContrastNormalization((0.75, 1.25)),
        iaa.Multiply((0.9, 1.1)),
        iaa.Grayscale(alpha=(0.0, 0.1))
    ]),
    iaa.OneOf([  # Either blur or noise transformation
        iaa.GaussianBlur(sigma=(0.0, 1.0)),
        iaa.AverageBlur(k=(2, 3)),
        iaa.MedianBlur(k=(3, 5)),
        iaa.BilateralBlur(d=(3, 5), sigma_color=(10, 50), sigma_space=(10, 50)),
        iaa.AdditiveGaussianNoise(scale=(0.0, 0.01 * 255)),
        iaa.ElasticTransformation(alpha=10, sigma=1)
    ])
])

/home/p.kuznetsov/runaway/venv/lib/python3.9/site-packages/imgaug/imgaug.py:188: DeprecationWarning: Function `ContrastNormalization()` is deprecated. Use `imgaug.contrast.LinearContrast` instead.
  warn_deprecated(msg, stacklevel=3)


In [4]:
image_paths = []
for root, dirs, files in os.walk(base_path):
    for file in files:
        if file.endswith(".jpg"):
            image_paths.append(os.path.join(root, file))

In [5]:
mask_paths = []
for root, dirs, files in os.walk(base_path):
    for file in files:
        if file.endswith(".txt"):
            mask_paths.append(os.path.join(root, file))

In [6]:
crops = []
for image_path in image_paths:
    image = Image.open(image_path)
    #image = add_padding_to_image(image, crop_size)
    width, height = image.size
    #mask_path = os.path.join(base_path, image_path.split("/")[2], "labels", os.path.basename(image_path).replace(".jpg", ".txt"))
    mask_path = os.path.join(base_path, "labels", os.path.basename(image_path).replace(".jpg", ".txt"))
    with open(mask_path, "r") as f:
        lines = f.readlines()
        empty = False
        if len(lines) == 0:
            empty = True
            x_min_valid = 0
            x_max_valid = width - crop_size
            y_min_valid = 0
            y_max_valid = height - crop_size
        else:
            line = lines[0]
            coordinates = line.split(" ")[1:]
            relative_x = [float(x) * width for x in coordinates[::2]]
            relative_y = [float(y) * height for y in coordinates[1::2]]

            x_min = min(relative_x)
            x_max = max(relative_x)
            y_min = min(relative_y)
            y_max = max(relative_y)

            x_min_valid = max(0, x_max - crop_size)
            x_max_valid = min(width - crop_size, x_min)
            y_min_valid = max(0, y_max - crop_size)
            y_max_valid = min(height - crop_size, y_min)

            if x_min_valid >= x_max_valid or y_min_valid >= y_max_valid or x_max_valid - x_min_valid < 0.5 or y_max_valid - y_min_valid < 0.5:
                print("Invalid crop for image " + image_path)
                continue

        crop_x = random.randint(math.ceil(x_min_valid), math.floor(x_max_valid))
        crop_y = random.randint(math.ceil(y_min_valid), math.floor(y_max_valid))

        crop = image.crop((crop_x, crop_y, crop_x + crop_size, crop_y + crop_size))
        
        if not empty:
            relative_x = [(x - crop_x) for x in relative_x]
            relative_y = [(y - crop_y) for y in relative_y]
            
            string = '0 '
            for x, y in zip(relative_x, relative_y):
                string += str(x / crop_size) + ' ' + str(y / crop_size) + ' '
        else:
            string = ''
        crops.append((crop, string))

Invalid crop for image ./roboflow_datasets/runaway-seg-12/train/images/104_2022-11-12_2022-12-22_png_jpg.rf.250d12cd0dcb07f0e1a4bca259b1a0ae.jpg
Invalid crop for image ./roboflow_datasets/runaway-seg-12/train/images/104_2022-02-07_2022-03-19_png_jpg.rf.a6dde812f91eb5293908fa4c555652d8.jpg
Invalid crop for image ./roboflow_datasets/runaway-seg-12/train/images/104_2022-11-12_2022-12-22_png_jpg.rf.28d04e9bfc7a7addceb119772c0cad3c.jpg
Invalid crop for image ./roboflow_datasets/runaway-seg-12/train/images/104_2022-07-16_2022-08-25_png_jpg.rf.0729771bccded21a521b472509eb3cf0.jpg
Invalid crop for image ./roboflow_datasets/runaway-seg-12/train/images/104_2022-06-06_2022-07-16_png_jpg.rf.7703e3c842389b97af1ef17a3b6db5ba.jpg
Invalid crop for image ./roboflow_datasets/runaway-seg-12/train/images/104_2022-06-06_2022-07-16_png_jpg.rf.f2a183f0bbd3b1dd5f243bccc9f0d538.jpg
Invalid crop for image ./roboflow_datasets/runaway-seg-12/train/images/104_2022-11-12_2022-12-22_png_jpg.rf.fb7a9f113294fd99bbf619

In [7]:
def get_polygon_from_line(line):
    data = line.strip().split()
    
    coords = list(map(float, data[1:]))
    
    points = [(coords[i], coords[i+1]) for i in range(0, len(coords), 2)]
    
    polygon = ia.Polygon(points)

    return polygon

In [8]:
def augment_image_and_polygons(image, line):
    image_np = np.array(image, dtype=np.uint8)
    
    polygons_on_image = ia.PolygonsOnImage([get_polygon_from_line(line)], shape=image_np.shape)
    
    image_aug_np, polygons_aug = seq(image=image_np, polygons=polygons_on_image)
    
    image_aug_pil = Image.fromarray(image_aug_np)
    
    return image_aug_pil, polygons_aug.polygons

In [9]:
for i in range(len(crops)):
    image, line = crops[i]
    try:
        image, polygons = augment_image_and_polygons(image, line)
    except:
        continue
    line = '0 '
    for x, y in polygons[0].exterior:
        line += str(x) + ' ' + str(y) + ' '
    crops[i] = (image, line)

In [10]:
train_size = int(len(crops) * train)
val_size = int(len(crops) * val)
test_size = int(len(crops) * test)

In [11]:
train_indices = random.sample(range(len(crops)), train_size)

remaining_indices = [i for i in range(len(crops)) if i not in train_indices]
val_indices = random.sample(remaining_indices, val_size)

test_indices = [i for i in range(len(crops)) if i not in train_indices and i not in val_indices]

train_crops = [crops[i] for i in train_indices]
val_crops = [crops[i] for i in val_indices]
test_crops = [crops[i] for i in test_indices]

In [12]:
print(len(image_paths))
print(len(crops))
print(len(train_crops))
print(len(val_crops))
print(len(test_crops))

3138
3116
2181
467
468


In [13]:
folder = "train"
os.makedirs(os.path.join(destination, folder, "images"), exist_ok=True)
os.makedirs(os.path.join(destination, folder, "labels"), exist_ok=True)
for image, data in train_crops:
    name = str(random.randint(0, 1000000))
    image.save(os.path.join(destination, folder, "images", name + ".jpg"))
    with open(os.path.join(destination, folder, "labels", name + ".txt"), "w") as f:
        f.write(data)

In [14]:
folder = "test"
os.makedirs(os.path.join(destination, folder, "images"), exist_ok=True)
os.makedirs(os.path.join(destination, folder, "labels"), exist_ok=True)
for image, data in test_crops:
    name = str(random.randint(0, 1000000))
    image.save(os.path.join(destination, folder, "images", name + ".jpg"))
    with open(os.path.join(destination, folder, "labels", name + ".txt"), "w") as f:
        f.write(data)

In [15]:
folder = "valid"
os.makedirs(os.path.join(destination, folder, "images"), exist_ok=True)
os.makedirs(os.path.join(destination, folder, "labels"), exist_ok=True)
for image, data in val_crops:
    name = str(random.randint(0, 1000000))
    image.save(os.path.join(destination, folder, "images", name + ".jpg"))
    with open(os.path.join(destination, folder, "labels", name + ".txt"), "w") as f:
        f.write(data)